In this notebook we are writing the code that gets all the tweets of the Key opinion leaders between 2014 - 2019
This is step 1
In step 2 we wil filter whose tweets 

In [3]:
from twitter_crawler import TwitterCrawler

my_token = 'AAAAAAAAAAAAAAAAAAAAADRFOwEAAAAAaTp%2Bdd1OhobYMYb5ExPXm7IL6RA%3DDHknH402gOXoegUGxNtpC6giIjdackRLtdRx6tjrnnLeFN1ntT'
my_twitter_crawler = TwitterCrawler(bearer_token= my_token)


In [4]:
kol_rawwaa_60_list = ["theresa_may",
"NicolaSturgeon",
"MichelBarnier",
"donaldtusk",
"fhollande",
"PhilipHammondUK",
"DavidDavisMP",
"JunckerEU",
"guyverhofstadt",
"DidierSeeuws",
"EndaKennyTD",
"hilarybennmp",
"MinPres",
"MartinSelmayr",
"Keir_Starmer",
"realDonaldTrump",
"Wolf_Schauble",
"LiamFox",
"TimmermansEU",
"BorisJohnson",
"Nigel_Farage",
"NJ_Timothy",
"DacrePaul",
"ManfredWeber",
"davidmcallister",
"matteorenzi",
"PJWheatcroft",
"SadiqKhan",
"marianorajoy",
"HeywoodFndation",
"MartinSchulz",
"cbicarolyn",
"rupertmurdoch",
"WeyandSabine",
"OllyRobbins",
"MLP_officiel",
"MarkJCarney",
"FrancesOGrady",
"giannipittella",
"MalmstromEU",
"TheGinaMiller",
"CER_Grant",
"shriti_vadera",
"carlosghosn"]

## Retrive tweets of Key Opinion Leaders
In the following code we are reading all the tweets of the list of the key opinion leaders in the given time-frame

In [6]:
#user_names=["theresa_may", "PhilipHammondUK", "MartinSelmayr", "BorisJohnson", "David_Cameron", "BSTimBarrow", "AMCarwyn", "WalkerWorcester"]
#user_names=["MichelBarnier"]
all_key_opinion_leader_names = kol_rawwaa_60_list
start_time = "2014-01-01T00:00:00Z"
end_time = "2019-01-01T00:00:00Z"
max_results = 500
limit_amount_of_returned_tweets = 10000000
dir_name = "KOL tweets"
##############################################################################################3
#### we don't want to get data on key opinion leaders that we already searched for their tweets!
# thus we first need to see which of the users that are inserted in the list are **not** in the files we already have:
from os import listdir
from os.path import isfile, join

import os.path
try:
    os.mkdir(dir_name)
    print("creating directory", dir_name, "to insert all the tables of all the key opinion leaders")
except:
    print("The dir", dir_name ,"already exist")

    
users_we_have_data = [f.split(".")[0] for f in listdir(dir_name) if isfile(join(dir_name, f))]

key_opinion_leader_names = list(set(all_key_opinion_leader_names) - set(users_we_have_data))
print("Users to search their tweets:",key_opinion_leader_names)
print("-----------------------------------------------")
print("Users we already have data on:",list(set(all_key_opinion_leader_names).intersection(users_we_have_data)))

# ##########################################
# my_twitter_crawler.return_tweets_of_key_opinion_leaders(query="", user_names= key_opinion_leader_names,
#                                         start_time = start_time,
#                                         end_time = end_time,
#                                         max_results = max_results, evaluate_last_token = False,
#                                         limit_amount_of_returned_tweets = limit_amount_of_returned_tweets,
#                                        verbose_10 = False, dir_name =dir_name)

The dir KOL tweets already exist
Users to search their tweets: ['carlosghosn', 'Wolf_Schauble', 'MarkJCarney', 'realDonaldTrump', 'NJ_Timothy', 'PJWheatcroft', 'shriti_vadera']
-----------------------------------------------
Users we already have data on: ['DavidDavisMP', 'FrancesOGrady', 'LiamFox', 'TimmermansEU', 'MartinSelmayr', 'DacrePaul', 'guyverhofstadt', 'NicolaSturgeon', 'EndaKennyTD', 'MinPres', 'theresa_may', 'donaldtusk', 'fhollande', 'Nigel_Farage', 'SadiqKhan', 'MalmstromEU', 'hilarybennmp', 'Keir_Starmer', 'matteorenzi', 'marianorajoy', 'WeyandSabine', 'MLP_officiel', 'TheGinaMiller', 'cbicarolyn', 'MichelBarnier', 'davidmcallister', 'BorisJohnson', 'giannipittella', 'PhilipHammondUK', 'rupertmurdoch', 'JunckerEU', 'DidierSeeuws', 'MartinSchulz', 'HeywoodFndation', 'OllyRobbins', 'ManfredWeber', 'CER_Grant']


In [7]:
key_opinion_leader_names

['carlosghosn',
 'Wolf_Schauble',
 'MarkJCarney',
 'realDonaldTrump',
 'NJ_Timothy',
 'PJWheatcroft',
 'shriti_vadera']

# Function filter tweets

We read numerous amount of tweets, comments, retweets and quotes from twitter. 
Now we wish to create a function that, given a dataframe with  tweets / comments / quotes it will enable scoring them.
The scoring will be based on the following parameters:

1. The author - if he is one of the KOPs - K points
2. The time - if it is in the day of key event - K points
3. The text - if it contains key words - K points

### Notes:
+ Note that you can't mix tables with tweets / comments / quotes - this is because they have different amount of columns. You need to process the tweets, comments and quotes table separately

+ **The key-events table and the KOP table - MUST BE OF XLSX TYPE!!**


### The Filter function gets as input:

+ 1. **dir_with_all_tweets** - The location / name of dir (if the dir is in the same location of the jupyter file) of all the tweets table that we want to read into the function (this is important as all the csv tweet tables that will be located in this location will be read into the function).

+ 2. **score_for_KOP** = By default = 5. This argument controls the **weight** given to tweet with author id of **Key Opinion Leader**

+ 3. **score_for_key_event** = By default = 5. This argument controls the **weight** given to tweet that was published in **key event day**.

+ 4. **score_for_key_words** = By default = 5. This argument controls the **weight** given to **each** **key-word** that is located in the tweet.

+ 5. **key_words** = By default = ["brexit", "eu", "deal"]. This argument controls the **Key words** to search in each tweet that are related to the brexit. Tweets with high amount of those words are assumed to be important.

+ 6. **threshold_score** - By default = 10. This is a very important argument. It controls the minimum score that a tweet must get in order to be returned in the filtered tweets table. Any tweet with score under this value won't be included in the filtered table

+ 7. **KOP_excel_name** - The name of the excel file (**xlsx format**) that contains the Key Opinion Leaders

+ 8. **key_events_excel_name** - The name of the excel file (**xlsx format**) that contains the key events data

+ 9. **stop_words_to_add** = A list with all the additional words you wish the function to handle as stop words (those words will be removed from each list of tokens for each tweet).

+ 10. **stop_words_file_name** = The name of the text file that contains all the stopwords. Note that if it doesn't exist, the function will go to "https://gist.githubusercontent.com/sebleier/554280/raw/7e0e4a1ce04c2bb7bd41089c9821dbcf6d0c786c/NLTK's%2520list%2520of%2520english%2520stopwords" - to find the stopwords

+ 11. **verbose** = by default = True. If True than the function will print its progress (suggested!)

### The function returns:

+ **tweets_table_filtered** - The filtered tweets table is a table containing all the tweets that their score is higher than the `threshold_score`

+ **tweets_table** - The preprocessed tweets table with all the tweets - This table is identical to the `tweets_table_filtered` except that it also included tweets with score under the `threshold_score`

+ **csv_files_evaluated** = a list with all the **csv** files that the function read and add to the tweets table (the files are read from the location you provided in the argument: `dir_with_all_tweets`)



In [2]:
from twitter_crawler import TwitterCrawler

my_token = 'AAAAAAAAAAAAAAAAAAAAADRFOwEAAAAAaTp%2Bdd1OhobYMYb5ExPXm7IL6RA%3DDHknH402gOXoegUGxNtpC6giIjdackRLtdRx6tjrnnLeFN1ntT'
my_twitter_crawler = TwitterCrawler(bearer_token= my_token)

In [5]:
files = [f for f in os.listdir('.') ]
for f in files:
    print(f)

.git
.gitattributes
.gitignore
All tweet data we have so far.ipynb
backup.py
brexit-env
Brexit_key_events.xlsx
Brexit_key_events2.csv
conversation_trees
count_proccesed_tweets.txt
count_requests_sent.txt
Get retweets of a certain tweet - roy madpis code.ipynb
key_opinion_leaders_tweets_tables_beta
KOP brexit.xlsx
LICENSE
README.md
requirments.txt
tweets
twitter_crawler.py
usage_examples.ipynb
venv
__pycache__


In [3]:
### inputs for the filter function
dir_with_all_tweets = os.path.join("key_opinion_leaders_tweets_tables_beta")

score_for_KOP = 5
score_for_key_event = 5
score_for_key_words = 5

key_words = ["brexit", "eu", "deal", "economy"]
threshold_score = 10

KOP_excel_name = "KOP brexit.xlsx"
key_events_excel_name = "Brexit_key_events.xlsx"
stop_words_to_add = ["https"]
#########################################################
tweets_table_filtered, tweets_table, csv_files_evaluated = my_twitter_crawler.filter_tweets_Brexit(
    dir_with_all_tweets = dir_with_all_tweets,
    score_for_KOP = score_for_KOP,
    score_for_key_event = score_for_key_event,
    score_for_key_words = score_for_key_words,
    key_words = key_words,
    threshold_score = threshold_score,
    KOP_excel_name = KOP_excel_name,
    key_events_excel_name = key_events_excel_name,
    stop_words_to_add = stop_words_to_add,
    stop_words_file_name = "stopwords.txt",
    verbose = True)

Step 1: Reading all the csv files
Step 2: Reading The Events Excel File
Step 3: Reading and preprocessing Key Opinion Leaders data
Step 4: Reading stop words data and adding your stop words

Step 5: Preprocess the tweets table for scoring:
Removing Stop words        | bigram, trigram, forthgram | merging the tweets table with the KOP and Events tables | 
found 31477 bigrams
found 29489 trigram
found 24101 forthgram

Finish preprocessing the tweets table, it took: 138.862 seconds
Step 6: Scoring the Tweets

Finish!
Total time: 189.798 Seconds ( 3.163 Minutes)


In [12]:
display(tweets_table_filtered.head())

,Unnamed: 0,author_id,conv_id_new,conversation_id,created_at,entities.mentions,id,id_new,public_metrics.like_count,public_metrics.quote_count,...,Index,Name,Born_in,twitter_user_name,Role,Place,clean_user_name,score_key_words,score_KOP,total_score
51801,431,1978270322,conv_id: 764361626212524032,764361626212524032,2016-08-13T07:23:07.000Z,"[{'start': 3, 'end': 12, 'username': 'worcsLEP...",764361626212524032,id: 764361626212524032,0,0,...,No-KOP,Robin Walker,England,@WalkerWorcester,No-KOP,No-KOP,WalkerWorcester,2,1,20
38206,439,17487476,conv_id: 1164940747847667714,1164940747847667714,2019-08-23T16:41:32.000Z,NaN,1164940747847667714,id: 1164940747847667714,130,6,...,1.0,Michel BARNIER,France,@MichelBarnier,Chief Negotiator for the,Commission Taskforce on Brexit Negotiations Eu...,MichelBarnier,3,1,20
10012,148,3131144855,conv_id: 1076417292139982849,1076417292139982849,2018-12-22T10:00:55.000Z,"[{'start': 3, 'end': 16, 'username': 'FraserNe...",1076417292139982849,id: 1076417292139982849,0,0,...,1.0,Boris JOHNSON,England,@BorisJohnson,Foreign Secretary,UK Government,BorisJohnson,3,1,20
4187,200,2476761393,conv_id: 859821415675174912,859821415675174912,2017-05-03T17:26:14.000Z,"[{'start': 0, 'end': 12, 'username': 'WelshLab...",859821415675174912,id: 859821415675174912,0,0,...,No-KOP,Carwyn Jones,Wales,@AMCarwyn,No-KOP,No-KOP,AMCarwyn,2,1,20
40969,265,2653613168,conv_id: 702594185942188033,702594185942188033,2016-02-24T20:41:22.000Z,NaN,702594185942188033,id: 702594185942188033,106,0,...,1.0,Philip HAMMOND,England,@PhilipHammondUK,Chancellor of the Exchequer,UK Government,PhilipHammondUK,3,1,20


In [8]:
tweets_table.shape

(56956, 41)

In [7]:
tweets_table_filtered.to_excel("tweets_table_filtered.xlsx")

### Which files are processed?

In [ ]:
print("csv files processed:\n")
for index, file in enumerate(csv_files_evaluated):
    print(index+1, file)

### See all the texts of the filtered tweets:

In [13]:
for i in tweets_table_filtered.text:
    print (i)
    print("-----------------------------------------")

RT @worcsLEP: Govt will cover cost of billions in EU subsidies for farming, science and deprived areas after Brexit.  Good news for worcs &amp;…
-----------------------------------------
Goed om #Brexit te bespreken met@MinPres. TA = best mogelijke deal, gebaseerd op UK rode lijnen. We zijn bereid om #UK-voorstellen te analyseren die realistisch, operationeel &amp; in lijn met onze principes zijn. EU wil orderlijke terugtrekking maar is voorbereid op elke uitkomst . https://t.co/gAiUfvxSMm
-----------------------------------------
RT @FraserNelson: The no-deal option: About a dozen Cabinet members now back completing Brexit on world trade terms rejecting the EU’s offe…
-----------------------------------------
@welshlabour So no deal is serious and would be felt by ordinary working people."No deal is better than a bad deal"? https://t.co/DF0nRSvChB
-----------------------------------------
Britain's #EU deal has been registered at the UN. Clear that the deal is legally binding &amp; ir